## Jane-Austen Dataset

refer: Project Gutenberg
https://xpmethod.columbia.edu/knowledge-design-studio/2019-10-26-corpus-db.html

In [ ]:
import json
import requests

In [ ]:
# create queries for Corpus-DB
baseURL = "http://corpus-db.org"

In [ ]:
def getTextAndMeta(author):
    metaResponse = requests.get(baseURL+"/api/author/"+author)
    textResponse = requests.get(baseURL+"/api/author/"+author+"/fulltext")
    meta = json.loads(metaResponse.text)
    texts = json.loads(textResponse.text)
    return meta, texts

In [ ]:
austenMeta, austenTexts = getTextAndMeta("Austen, Jane")

In [ ]:
len(austenMeta), len(austenTexts)

In [ ]:
[(book['id'], book['title']) for book in austenMeta]

In [ ]:
# keeping only english texts
myAustenCollection = [105, 121, 141, 158, 161, 946, 1212, 1342]

In [ ]:
austenMetaSubset = [book for book in austenMeta if int(float(book['id'])) in myAustenCollection]
austenTextSubset = [book for book in austenTexts if int(book['id']) in myAustenCollection]

In [ ]:
len(austenMetaSubset), len(austenTextSubset)

In [ ]:
combined_text = ""
for book in austenTextSubset:
    combined_text += book['text']

In [ ]:
import re

# Replace sequences of one or more whitespace characters with a single space
whitespace_normalized_text = re.sub(r'\s+', ' ', combined_text)

# Strip leading and trailing whitespace
whitespace_normalized_text = whitespace_normalized_text.strip()

print(f"Length of combined_text: {len(combined_text)}")
print(f"Length of whitespace_normalized_text: {len(whitespace_normalized_text)}")
print(f"First 500 characters of whitespace_normalized_text:\n{whitespace_normalized_text[:500]}")

In [ ]:
cleaned_text = whitespace_normalized_text

# Remove specific introductory/concluding phrases
cleaned_text = re.sub(r'by Al Haines\.', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'Finis', '', cleaned_text, flags=re.IGNORECASE)

# Remove Project Gutenberg disclaimers and similar meta-information
cleaned_text = re.sub(r'Project Gutenberg-tm is synonymous with the free distribution of electronic texts.', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'The Project Gutenberg EBook of [^\.]+\. This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever\. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www\.gutenberg\.org', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'ADVERTISEMENT BY THE AUTHORESS(?:, TO THE SECOND EDITION)?', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'End of the Project Gutenberg EBook of [^\.]+\.', '', cleaned_text, flags=re.IGNORECASE)

# Remove book titles and author names that appear as headers/footers but are not part of narrative
# This pattern needs to be more careful, but for now, focus on clear header-like structures
cleaned_text = re.sub(r'Persuasion by Jane Austen \(\d{4}\)', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'NORTHANGER ABBEY by Jane Austen \(\d{4}\)', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'MANSFIELD PARK \(\d{4}\) By Jane Austen', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'EMMA by Jane Austen \(\d{4}\)', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'SENSE AND SENSIBILITY by Jane Austen \(\d{4}\)', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'Lady Susan by Jane Austen \(\d{4}\)', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'LOVE AND FREINDSHIP \[sic\] by Jane Austen \(\d{4}\)', '', cleaned_text, flags=re.IGNORECASE)
cleaned_text = re.sub(r'PRIDE AND PREJUDICE By Jane Austen', '', cleaned_text, flags=re.IGNORECASE)


# Further normalize whitespace after removals
cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

print(f"Length of whitespace_normalized_text: {len(whitespace_normalized_text)}")
print(f"Length of cleaned_text: {len(cleaned_text)}")
print(f"First 500 characters of cleaned_text:\n{cleaned_text[:500]}")


In [ ]:
output_filename = "austen_combined_cleaned.txt"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"Cleaned text saved to {output_filename}")

## Bigram Language Model

In [ ]:
with open('austen_combined_cleaned.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

In [ ]:
# look at the first 1000 characters
print(text[:1000])

In [ ]:
# unique characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

In [ ]:
# encoding the entire data set and storing into Torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earlier will to the GPT look like this

In [ ]:
# train-val split
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

In [ ]:
# input to the model
print(xb)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


In [ ]:
# sampling from the output
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

In [ ]:
# implementing transformers
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# using our Austen Jane Dataset
with open('austen_combined_cleaned.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
